# Construcción de dataset de recuperaciones en campo abierto
Usamos `events_all_37_90.csv` y construiremos un dataset de posesiones que empiezan en recuperación, etiquetadas según si el equipo que recupera consigue un tiro propio.

## Imports, paths y configuración

In [ ]:
import os
from pathlib import Path
import re
import numpy as np
import pandas as pd

BASE_DIR = Path().resolve()
DATA_PATH = BASE_DIR / "source" / "events_all_37_90.csv"
TIME_WINDOW_S = 60
XBINS, YBINS  = 12, 8   
K_LOOKBACK    = 8      
NUMERIC_KEEP = ["pass_length", "pass_angle"]
CAT_KEEP = ["play_pattern", "pass_height", "pass_type", "pass_outcome","dribble_outcome", "duel_type", "duel_outcome"]
FLAG_KEEP = ["under_pressure", "counterpress", "dribble_overrun", "dribble_no_touch","pass_cross", "pass_cut_back", "pass_switch", "pass_through_ball", "pass_straight"]

print("Directorio base:", BASE_DIR)
print("Ruta de eventos:", DATA_PATH)


## Cargamos los eventos del dataset y definimos tiempos

In [ ]:
df = pd.read_csv(DATA_PATH, low_memory=False)

df = (df.sort_values(["match_id", "period", "index"], kind="mergesort").reset_index(drop=True))
for c in ["match_id", "period", "team_id", "possession",
          "possession_team_id", "player_id", "index"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df["minute"] = pd.to_numeric(df["minute"], errors="coerce")
df["second"] = pd.to_numeric(df["second"], errors="coerce")
df["t"] = df["minute"] * 60 + df["second"]
df.head()


## Definición de coordenadas x,y en el campo

In [ ]:

float_re = re.compile(r"[-+]?(?:\d*\.\d+|\d+\.?)(?:[eE][-+]?\d+)?")

def parse_xy(val):
    """Extrae (x, y) desde lista/tupla o string; devuelve NaN si falla."""
    if isinstance(val, (list, tuple)) and len(val) >= 2:
        return float(val[0]), float(val[1])
    if isinstance(val, str):
        nums = float_re.findall(val)
        if len(nums) >= 2:
            try:
                return float(nums[0]), float(nums[1])
            except Exception:
                return (np.nan, np.nan)
    return (np.nan, np.nan)

if "x" not in df.columns or "y" not in df.columns:
    xy = df.get("location", pd.Series([None] * len(df))).apply(parse_xy)
    df["x"] = [a for a, b in xy]
    df["y"] = [b for a, b in xy]

print("Columnas de coordenadas disponibles:", [c for c in ["x", "y"] if c in df.columns])
df[["x", "y"]].head()


## Identificamos y quitamos a las arqueras

In [ ]:
pos_str = df.get("position", pd.Series(index=df.index, dtype="object")).astype(str).str.lower()
df["is_gk_event"] = pos_str.str.contains("goalkeeper", na=False)
is_gk_by_player = df.groupby("player_id")["is_gk_event"].max().rename("is_gk_player")
df["is_gk_player"] = df["player_id"].map(is_gk_by_player).fillna(False)
df["is_gk"] = df["is_gk_event"] | df["is_gk_player"]
df = df.drop(columns=[c for c in df.columns if c.startswith("goalkeeper_")], errors="ignore")
print("Marcadores de portero creados. Ejemplo:")
df[["player_id", "position", "is_gk_event", "is_gk_player", "is_gk"]].head()


## Resumen por posesión e identificación de recuperaciones

In [ ]:
gpos = df.groupby(["match_id", "period", "possession"], sort=False, observed=True)
pos_core = gpos.agg(
    start_index=("index", "min"),
    start_time=("t", "min"),
    pos_team=("possession_team_id", "first")
).reset_index()
pos_tbl = pos_core.sort_values(["match_id", "period", "possession"], kind="mergesort")
pos_tbl["prev_pos_team"] = pos_tbl.groupby(["match_id", "period"])["pos_team"].shift(1)
recoveries = pos_tbl[
    (pos_tbl["prev_pos_team"].notna()) &
    (pos_tbl["prev_pos_team"] != pos_tbl["pos_team"])
].copy()

print("Posesiones totales:", len(pos_tbl))
print("Posesiones que empiezan en recuperación (cambio de equipo):", len(recoveries))
recoveries.head()


## Tiros propios tras la recuperación

In [ ]:
shots = df[df["type"].eq("Shot")][["match_id", "period", "team_id", "t"]].rename(columns={"team_id": "shot_team_id", "t": "t_shot"})

def first_own_shot_after(gt: pd.DataFrame, gs: pd.DataFrame) -> pd.DataFrame:
    if gt.empty:
        return gt.assign(t_shot=np.nan, shot_team_id=np.nan)
    if gs.empty:
        return gt.assign(t_shot=np.nan, shot_team_id=np.nan)

    gt = gt.sort_values("start_time", kind="mergesort").reset_index(drop=True)
    gs = gs.sort_values("t_shot", kind="mergesort").reset_index(drop=True)
    t0   = gt["start_time"].to_numpy()
    team = pd.to_numeric(gt["pos_team"], errors="coerce").to_numpy()
    tS   = gs["t_shot"].to_numpy()
    tmS  = pd.to_numeric(gs["shot_team_id"], errors="coerce").to_numpy()
    idx = np.searchsorted(tS, t0, side="left")
    out_t  = np.full(len(gt), np.nan)
    out_tm = np.full(len(gt), np.nan)
    for i in range(len(gt)):
        j = idx[i]
        while j < len(tS) and (tmS[j] != team[i] or np.isnan(tmS[j])):
            j += 1
        if j < len(tS):
            out_t[i]  = tS[j]
            out_tm[i] = tmS[j]

    return gt.assign(t_shot=out_t, shot_team_id=out_tm)

res = []
g_rec = recoveries.groupby(["match_id", "period"], sort=False)
gshot = shots.groupby(["match_id", "period"], sort=False)

for key, gt in g_rec:
    gs = gshot.get_group(key) if key in gshot.groups else pd.DataFrame(columns=["t_shot", "shot_team_id"])
    res.append(first_own_shot_after(gt, gs))

rec_df = (
    pd.concat(res, ignore_index=True)
    if res else
    pd.DataFrame(columns=list(recoveries.columns) + ["t_shot", "shot_team_id"])
)

rec_df["dt"] = rec_df["t_shot"] - rec_df["start_time"]
rec_df["y_shot"] = ((rec_df["dt"] >= 0) & (rec_df["dt"] <= TIME_WINDOW_S)).astype(int)

print("Recuperaciones etiquetadas:", len(rec_df),"| Positivas (tiro propio <= %ds):" % TIME_WINDOW_S, int(rec_df["y_shot"].sum()))
rec_df[["match_id", "period", "possession", "dt", "y_shot"]].head()


## Construcción de k eventos en el campo

In [ ]:
cols_for_anchor = ["match_id", "period", "possession", "index","team_id", "t", "minute", "x", "y", "type"]
cols_for_anchor += [c for c in NUMERIC_KEEP if c in df.columns]
cols_for_anchor += [c for c in CAT_KEEP if c in df.columns]
cols_for_anchor += [c for c in FLAG_KEEP if c in df.columns]
outfield = df[~df["is_gk"]][cols_for_anchor].copy()
anchors = []
for (mid, per, pos), g in outfield.groupby(["match_id", "period", "possession"], sort=False):
    g = g.sort_values("index")
    head = g.head(K_LOOKBACK)
    if head.empty:
        continue
    anchors.append(head)

anchors = (
    pd.concat(anchors, ignore_index=True)
    if anchors else
    pd.DataFrame(columns=cols_for_anchor)
)
anchors.head()


## Filtro de eventos y selección de feautures

In [ ]:
anchor_tbl = anchors.merge(
    rec_df[["match_id", "period", "possession", "y_shot"]],
    on=["match_id", "period", "possession"],
    how="inner"   # solo posesiones que empiezan tras recuperación
)

print("Anchors tras filtrar a posesiones de recuperación:", len(anchor_tbl))
# Construcción de features del anchor
anchor_tbl = anchor_tbl.rename(
    columns={
        "x": "ax",
        "y": "ay",
        "minute": "a_minute",
        "type": "a_type"
    }
)
anchor_tbl.head()


## Bins espaciales y zone_id

In [ ]:
xbins = np.linspace(0, 120, XBINS + 1)
ybins = np.linspace(0, 80, YBINS + 1)

anchor_tbl["zx"] = pd.cut(anchor_tbl["ax"], xbins, labels=False, include_lowest=True)
anchor_tbl["zy"] = pd.cut(anchor_tbl["ay"], ybins, labels=False, include_lowest=True)
anchor_tbl["zone_id"] = (
    anchor_tbl["zx"].astype("Int64").astype(str) +
    "_" +
    anchor_tbl["zy"].astype("Int64").astype(str)
)
anchor_tbl[["ax", "ay", "zx", "zy", "zone_id", "a_minute", "a_type", "y_shot"]].head()

## Selección de columnas finales

In [ ]:
final_cols = [
    "match_id", "period", "possession", "team_id",
    "ax", "ay", "zx", "zy", "zone_id",
    "a_minute", "a_type", "y_shot"
]
final_cols += [c for c in NUMERIC_KEEP if c in anchor_tbl.columns]
final_cols += [c for c in CAT_KEEP if c in anchor_tbl.columns]
final_cols += [c for c in FLAG_KEEP if c in anchor_tbl.columns]
df_train = anchor_tbl[final_cols].copy()
num_cols = ["ax", "ay", "zx", "zy", "a_minute"] + [
    c for c in NUMERIC_KEEP if c in df_train.columns
]
for c in num_cols:
    df_train[c] = pd.to_numeric(df_train[c], errors="coerce").fillna(-1)

for c in [c for c in CAT_KEEP + ["a_type"] if c in df_train.columns]:
    df_train[c] = df_train[c].astype(str)
    df_train.loc[
        df_train[c].isin(["nan", "None", "NaN"]) | df_train[c].isna(),
        c
    ] = "Missing"
for c in [c for c in FLAG_KEEP if c in df_train.columns]:
    df_train[c] = pd.to_numeric(df_train[c], errors="coerce").fillna(0).astype(int)

df_train["y_shot"] = df_train["y_shot"].fillna(0).astype(int)

print("Recuperaciones únicas (posesiones):", len(rec_df))
print("K_LOOKBACK:", K_LOOKBACK,
      "=> filas esperadas ≈", len(rec_df) * K_LOOKBACK)
print("Filas finales reales:", len(df_train),
      "| Positivas:", int(df_train["y_shot"].sum()))

df_train.head()


## Guardado del csv

In [ ]:
OUTPUT_DIR = BASE_DIR / "source"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
out_csv_name = f"train_recoveries_K{K_LOOKBACK}_T{TIME_WINDOW_S}s.csv"
out_csv_path = OUTPUT_DIR / out_csv_name
df_train.to_csv(out_csv_path, index=False)

print("Dataset de recuperaciones guardado en:", out_csv_path)
out_csv_path
